In [1]:
class memoize(object): # pylint: disable=invalid-name
    """Decorator to memoize a function"""
    def __init__(self, func):
        self.func = func
        self.cache = {}
        
    def __get__(self, instance, owner):
        self.parent = instance
        print('GET', instance, owner)
        return self.__call__

    def __call__(self, *args):
        if args not in self.cache:
            self.cache[args] = self.func(*args)
        return self.cache[args]
    
@memoize
def foobar(ducks=10, guns=0):
    return ducks - guns


print(foobar.__dict__)

remaining = foobar(10, 3)
print(remaining)

print(foobar.__dict__)

{'cache': {}, 'func': <function foobar at 0x7f08f9486de8>}
7
{'cache': {(10, 3): 7}, 'func': <function foobar at 0x7f08f9486de8>}


In [2]:
class memoize(object): # pylint: disable=invalid-name
    """Decorator to memoize a function"""
    def __init__(self, func):
        self.func = func
        self.cache = {}
        
    def __get__(self, instance, owner):
        self.parent = instance
        print('GET', instance, owner)
        return super(memoize,self).__get__(self, instance, owner)

    def __call__(self, *args):
        print('CALL', self.__dict__, args)
        if args not in self.cache:
            print('CACHING', args)
            if hasattr(self, 'parent'):
                self.cache[args] = self.func(self.parent, *args)
            else:
                self.cache[args] = self.func(*args)
        return self.cache[args]
    
class DUCKS:
    
    def __init__(self, ducks):
        self.ducks = ducks

    @memoize
    def hunt(self, guns=1):
        return self.ducks - guns

@memoize
def foobaz(ducks, guns):
    return ducks - guns

print('foobaz dict', foobaz.__dict__)
remaining = foobaz(10, 3)
print(remaining)
print('foobaz dict', foobaz.__dict__)
print('foobaz', foobaz)

x = DUCKS(20)
print('x dict', x.hunt.__dict__, x.__dict__)
r = x.hunt(4)
print(r)
print('x dict', x.hunt.__dict__, x.__dict__)
r = x.hunt(4)
print(r)
print('x hunt', x.hunt)

('foobaz dict', {'cache': {}, 'func': <function foobaz at 0x7f08fdfd4668>})
('CALL', {'cache': {}, 'func': <function foobaz at 0x7f08fdfd4668>}, (10, 3))
('CACHING', (10, 3))
7
('foobaz dict', {'cache': {(10, 3): 7}, 'func': <function foobaz at 0x7f08fdfd4668>})
('foobaz', <__main__.memoize object at 0x7f08f8c31bd0>)
('GET', <__main__.DUCKS instance at 0x7f08f8c40e60>, <class __main__.DUCKS at 0x7f08f950a870>)


TypeError:  expected at most 2 arguments, got 3

In [ ]:
def check_authorization(f):
    def wrapper(*args):
        print args[0].url
        return f(*args)
    return wrapper

class Client(object):
    def __init__(self, url):
        self.url = url

    @check_authorization
    def get(self):
        print 'get'

Client('http://www.google.com').get()

In [ ]:
class memoize(object): # pylint: disable=invalid-name
    """Decorator to memoize a function"""
    def __init__(self, func):
        self.func = func
        self.cache = {}
        print('INIT', func)

    def __call__(self, *args):
        print('CALL', self.__dict__, args)
        if args not in self.cache:
            print('CACHING', args)
            if hasattr(self, 'parent'):
                self.cache[args] = self.func(self.parent, *args)
            else:
                self.cache[args] = self.func(*args)
        return self.cache[args]
    
class DUCKS:
    
    def __init__(self, ducks):
        self.ducks = ducks

    @memoize
    def hunt(self, guns=1):
        return self.ducks - guns

@memoize
def foobaz(ducks, guns):
    return ducks - guns

print('foobaz dict', foobaz.__dict__)
remaining = foobaz(10, 3)
print(remaining)
print('foobaz dict', foobaz.__dict__)
print('foobaz', foobaz)

x = DUCKS(20)
print('x dict', x.hunt.__dict__, x.__dict__)
r = x.hunt(4)
print(r)
print('x dict', x.hunt.__dict__, x.__dict__)
r = x.hunt(4)
print(r)
print('x hunt', x.hunt)

In [ ]:
def memoize(f):
    def wrapper(*args):
        print(args)
        return f(*args)
    return wrapper


class DUCKS:
    
    def __init__(self, ducks):
        self.ducks = ducks

    @memoize
    def hunt(self, guns=1):
        return self.ducks - guns

    
d = DUCKS(12)
r = d.hunt(4)
print(r)

In [ ]:
def memoize(f):
    print('INIT func', f.__dict__)
    
    def wrapper(*args):
        print('ARGS', args)
        cache = getattr(f, 'cache', {})
        print('CACHE', cache)
        result = f(*args)
        if args not in cache:
            cache[args] = result
            setattr(f, 'cache', cache)
            
        return result
    return wrapper


class DUCKS:
    
    def __init__(self, ducks):
        print('INIT DUCKS', ducks)
        self.ducks = ducks
    
    @property
    @memoize
    def count(self):
        return self.ducks

    @memoize
    def hunt(self, guns=1):
        return self.ducks - guns

@memoize
def foobar(ducks, guns):
    return ducks - guns

    

print('---FOOBAR---')
r = foobar(12, 3)
print('result:', r)
print('dict:', foobar.__dict__)
print()

print('---FOOBAR---')
r = foobar(12, 3)
print('result:', r)
print('dict:', foobar.__dict__)
print()

print('---COUNT---')
d = DUCKS(32)
r = d.count
print('result:', r)
print('obj:', dir(d))
print('dict:', d.__dict__)
print()
print('---COUNT---')
d = DUCKS(32)
r = d.count
print('result:', r)
print('obj:', dir(d))
print('dict:', d.__dict__)
print()

print('---HUNT---')
d = DUCKS(50)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()
print('---HUNT---')
d = DUCKS(50)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()


In [ ]:
def class_memoize(f):
    def wrapper(*args):
        cache = getattr(args[0], '_cache', {})
        key = (f.func_name, args[1:])
        if key not in cache:
            cache[key] = f(*args)
            setattr(args[0], '_cache', cache)
        return cache[key]
    return wrapper


class DUCKS:
    
    def __init__(self, ducks):
        print('INIT DUCKS', ducks)
        self.ducks = ducks
    
    @property
    @class_memoize
    def count(self):
        return self.ducks

    @class_memoize
    def hunt(self, guns=1):
        return self.ducks - guns

print('---COUNT---')
d = DUCKS(32)
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---COUNT---')
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(13)
print('result:', r)
print('dict:', d.__dict__)
print()

In [ ]:
from functools import wraps

def memoize(f=None, **ARGS):
    print('init decorator',f, ARGS)
    # save cache in the parent object
    def class_memoize(f):
        print('init cls', f)
        # wrapper class
        def wrapper(*args):
            print('args', args)
            cache = getattr(args[0], '_cache', {})
            key = (f.func_name, args[1:])
            if key not in cache:
                cache[key] = f(*args)
                setattr(args[0], '_cache', cache)
            return cache[key]
        return wrapper
    # save cache in the function object itself
    def func_memoize(f):
        print('init func', f)
        cache = getattr(f, 'cache', {})
        print('FM CACHE', cache, f)
        
        @wraps(f)
        def wrapper(*args):
            print('ARGS', args)
            cache = getattr(f, 'cache', {})
            print('CACHE', cache, f.__dict__)
            print('inside wrapper', f, )
            result = f(*args)
            if args not in cache:
                cache[args] = result
                setattr(f, 'cache', cache)
            return result
        return wrapper
    # decide which one to give
    if f is not None:
        print('CLASS MEMOIZE')
        return class_memoize(f)
    print('FUNC MEMOIZE')
    return func_memoize


class DUCKS:
    
    def __init__(self, ducks):
        print('INIT DUCKS', ducks)
        self.ducks = ducks
    
    @property
    @memoize
    def count(self):
        return self.ducks

    @memoize
    def hunt(self, guns=1):
        return self.ducks - guns

    
@memoize(func=True)
def foobar(ducks, guns):
    return ducks - guns

print('---COUNT---')
d = DUCKS(32)
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---COUNT---')
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(13)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---FOOBAR---')
print('dict:', foobar.func_closure[0].cell_contents.__dict__)
r = foobar(12, 3)
print('result:', r)
print('foobar:', foobar)
print('dict:', foobar.func_closure[0].cell_contents.__dict__)
print()

print('---FOOBAR---')
print('dict:', foobar.func_closure[0].cell_contents.__dict__)
r = foobar(12, 3)
print('result:', r)
print('foobar:', foobar)
print('dict:', foobar.func_closure[0].cell_contents.__dict__)
print()

In [11]:
import inspect
from inspect import signature

def decorate(f):
    print('func:', f)
#     print('func class:', f.__class__)
#     print('func dir:', dir(f))
#     print('is func:', inspect.isfunction(f))
#     print('is method:', inspect.ismethod(f))
#     print('f call:', f.__call__)
    sig = signature(f)
    print('sig:', sig)
    print('sig:', sig.parameters.keys())
    first_param = None
    if len(sig.parameters) > 0:
        first_param = list(sig.parameters.keys())[0]
    print('0 param:', first_param)
#     print('func func_globals:', f.func_globals)
#     print('func func_code:', f.func_code)
#     print('func func_doc:', f.func_doc)
    
    
    def wrapper(*args):
        print('inside wrapper:', f, args)
        result =  f(*args)
        print('result:', result)
        return result
    return wrapper


class DUCKS:
    
    def __init__(self, ducks):
        print('INIT DUCKS', ducks)
        self.ducks = ducks
    
    @property
    @decorate
    def count(self):
        return self.ducks

    @decorate
    def hunt(self, guns=1):
        return self.ducks - guns

@decorate    
def foobar(ducks, guns):
    return ducks - guns

@decorate 
def foobaz():
    return 42

d = DUCKS(20)

print('--COUNT--')
x = d.count
print('X', x)
print('type:', type(d.count))
print('dir:', dir(d))
print('class:', d.count.__class__)
print('type mod:', type(d.__module__))
print()

print('--HUNT--')
x = d.hunt(3)
print('X', x)
print('type:', type(d.hunt))
print()

print('--FOOBAR--')
x = foobar(12,3)
print('X', x)
print('type:', type(foobar))
print('class:', foobar.__class__)
print()


print('--FOOBAZ--')
x = foobaz()
print('X', x)
print('type:', type(foobaz))
print('class:', foobaz.__class__)
print()



func: <function DUCKS.count at 0x7f56e80f5bf8>
sig: (self)
sig: odict_keys(['self'])
0 param: self
func: <function DUCKS.hunt at 0x7f56e80f5598>
sig: (self, guns=1)
sig: odict_keys(['self', 'guns'])
0 param: self
func: <function foobar at 0x7f56e80f5d90>
sig: (ducks, guns)
sig: odict_keys(['ducks', 'guns'])
0 param: ducks
func: <function foobaz at 0x7f56e80f5c80>
sig: ()
sig: odict_keys([])
0 param: None
INIT DUCKS 20
--COUNT--
inside wrapper: <function DUCKS.count at 0x7f56e80f5bf8> (<__main__.DUCKS object at 0x7f56e803b4e0>,)
result: 20
X 20
inside wrapper: <function DUCKS.count at 0x7f56e80f5bf8> (<__main__.DUCKS object at 0x7f56e803b4e0>,)
result: 20
type: <class 'int'>
dir: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__

In [24]:
from functools import wraps
from inspect import signature

def memoize(f):
    print('init decorator',f)
    
    def get_first_param(f):
        sig = signature(f)
        if len(sig.parameters) < 1:
            return None
        return list(sig.parameters.keys())[0]
    
    fp = get_first_param(f)
    # save cache in the parent object
    def class_memoize(f):
        print('init cls', f)
        # wrapper class
        @wraps(f)
        def wrapper(*args):
            print('args', args)
            cache = getattr(args[0], '_cache', {})
            print('F', f, f.__name__)
            key = (f.__name__, args[1:])
            if key not in cache:
                cache[key] = f(*args)
                setattr(args[0], '_cache', cache)
            return cache[key]
        return wrapper
    # save cache in the function object itself
    def func_memoize(f):
        print('init func', f)
        cache = getattr(f, 'cache', {})
        print('FM CACHE', cache, f)
        
        @wraps(f)
        def wrapper(*args):
            print('ARGS', args)
            cache = getattr(f, 'cache', {})
            print('CACHE', cache, f.__dict__)
            print('inside wrapper', f, )
            result = f(*args)
            if args not in cache:
                cache[args] = result
                setattr(f, 'cache', cache)
            return result
        return wrapper
    # decide which one to give
    if fp == 'self':
        print('CLASS MEMOIZE')
        return class_memoize(f)
    print('FUNC MEMOIZE')
    return func_memoize(f)

    
    

class DUCKS:
    
    def __init__(self, ducks):
        print('INIT DUCKS', ducks)
        self.ducks = ducks
    
    @property
    @memoize
    def count(self):
        return self.ducks

    @memoize
    def hunt(self, guns=1):
        return self.ducks - guns

@memoize    
def foobar(ducks, guns):
    return ducks - guns

@memoize 
def foobaz():
    return 42


d = DUCKS(20)

print('---COUNT---')
d = DUCKS(32)
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---COUNT---')
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(13)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---FOOBAR---')
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
r = foobar(12, 3)
print('result:', r)
print('foobar:', foobar)
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
print()

print('---FOOBAR---')
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
r = foobar(12, 3)
print('result:', r)
print('foobar:', foobar)
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
print()


print('---FOOBAZ---')
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
r = foobaz()
print('result:', r)
print('foobar:', foobaz)
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
print()


print('---FOOBAZ---')
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
r = foobaz()
print('result:', r)
print('foobar:', foobaz)
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
print()

init decorator <function DUCKS.count at 0x7f56e80a7400>
CLASS MEMOIZE
init cls <function DUCKS.count at 0x7f56e80a7400>
init decorator <function DUCKS.hunt at 0x7f56e80f5c80>
CLASS MEMOIZE
init cls <function DUCKS.hunt at 0x7f56e80f5c80>
init decorator <function foobar at 0x7f56e80a7378>
FUNC MEMOIZE
init func <function foobar at 0x7f56e80a7378>
FM CACHE {} <function foobar at 0x7f56e80a7378>
init decorator <function foobaz at 0x7f56e80a7048>
FUNC MEMOIZE
init func <function foobaz at 0x7f56e80a7048>
FM CACHE {} <function foobaz at 0x7f56e80a7048>
INIT DUCKS 20
---COUNT---
INIT DUCKS 32
dict: {'ducks': 32}
args (<__main__.DUCKS object at 0x7f56dafbc4e0>,)
F <function DUCKS.count at 0x7f56e80a7400> count
result: 32
dict: {'ducks': 32, '_cache': {('count', ()): 32}}

---COUNT---
dict: {'ducks': 32, '_cache': {('count', ()): 32}}
args (<__main__.DUCKS object at 0x7f56dafbc4e0>,)
F <function DUCKS.count at 0x7f56e80a7400> count
result: 32
dict: {'ducks': 32, '_cache': {('count', ()): 32}}
